In [1]:
import pandas as pd
import numpy as np
import warnings

from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score

from sys import platform

if platform == "win32":
    path = 'C:/Users/olive/GitHub/f1-analytics/'
elif platform == "darwin":
    path = '~/Documents/GitHub/f1-analytics/'
    # path = '/Users/oliverjcarter/Documents/GitHub/f1-analytics/'

warnings.filterwarnings("ignore", category=RuntimeWarning) 
pd.options.mode.chained_assignment = None  # default='warn'

%matplotlib inline

In [2]:
data = pd.read_csv(path+'data/ml_input.csv')
processed = pd.read_csv(path+'data/processed.csv')
parameters = pd.read_csv(path+'parameters/svm_classifier.csv').drop('Unnamed: 0', axis=1)

In [3]:
### Season to test results

N = 2021

In [5]:
df = data.copy()
df.podium = df.podium.map(lambda x: 1 if x == 1 else 0)

train = df[df.season < N]

scaler = StandardScaler()
X_train = train.drop([
        'season', 'round',
        'podium'], axis=1)
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)

y_train = np.asarray(train.podium.values)

In [6]:
def score_classification(model):
    predictions = []
    
    for circuit in df[df.season == N]['round'].unique():

        test = df[(df.season == N) & (df['round'] == circuit)]
        X_test = test.drop([
            'season', 'round',
            'podium'], axis=1)
        y_test = test.podium

        # Scaling
        X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

        # Make Predictions
        prediction_df = pd.DataFrame(model.predict_proba(X_test), columns=['proba_0', 'proba_1'])
        prediction_df['actual'] = y_test.reset_index(drop=True)
        prediction_df['predicted'] = prediction_df.proba_1.map(lambda x: 1 if x == prediction_df.proba_1.max() else 0)


        predictions += list(prediction_df['predicted'].values)

    return predictions

In [27]:
params = parameters.iloc[1]

gamma = params.gamma
c = params.C
kernel = params.kernel

params

model     svm_classifier
gamma           0.000207
C               0.061585
kernel               rbf
score           0.681818
Name: 1, dtype: object

In [28]:
model = svm.SVC(probability=True, gamma=gamma, C=c, kernel=kernel)
model.fit(X_train, y_train)

predictions = score_classification(model)

In [29]:
df = processed[['season', 'round', 'driver', 'starting_grid', 'podium', 'driver_points_per_race', 'driver_points_last_3_races', 'q_delta']].query('season == @N')

df['predicted'] = predictions

In [33]:
df

,season,round,driver,starting_grid,podium,driver_points_per_race,driver_points_last_3_races,q_delta,predicted
3188,2022,1,charles_leclerc,1,1,0.00,0.0,0.000,0
3189,2022,1,carlos_sainz,3,2,0.00,0.0,0.129,0
3190,2022,1,lewis_hamilton,5,3,0.00,0.0,0.680,1
3191,2022,1,george_russell,9,4,0.00,0.0,1.658,0
3192,2022,1,kevin_magnussen,7,5,0.00,0.0,1.250,0
...,...,...,...,...,...,...,...,...,...
3614,2022,22,mick_schumacher,12,16,0.55,0.0,1.401,0
3615,2022,22,kevin_magnussen,16,17,1.14,3.0,2.010,0
3616,2022,22,lewis_hamilton,5,18,10.91,60.0,0.684,1
3617,2022,22,nicholas_latifi,20,19,0.09,0.0,2.230,0
